In [ ]:
import pandas as pd
import glob
!pip install boto3 s3fs
import boto3
import pandas as pd
import io


In [ ]:
s3 = boto3.client('s3',
    aws_access_key_id='YOUR_ACCESS_KEY_ID',
    aws_secret_access_key='YOUR_SECRET_ACCESS_KEY',
    region_name='us-east-1'
)



In [ ]:
bucket_name = "movie-reco-raw-nishu"
prefix = "Raw/"

response = s3.list_objects_v2(Bucket=bucket_name, Prefix=prefix)

files = [obj['Key'] for obj in response.get('Contents', [])]
print("Files in S3 final folder:")
for f in files:
    print(f)


In [ ]:
import pandas as pd

# Read ratings.csv directly from S3
ratings_file = "Raw/ratings.csv"

# Use boto3 to get the file object
obj = s3.get_object(Bucket=bucket_name, Key=ratings_file)
ratings = pd.read_csv(obj['Body'])

# Quick check
ratings.head()
ratings.info()


In [ ]:
# Read ratings.csv directly from S3
movies_file = "Raw/movies.csv"

# Use boto3 to get the file object
obj = s3.get_object(Bucket=bucket_name, Key=movies_file)
movies = pd.read_csv(obj['Body'])

# Quick check
movies.head()
movies.info()

In [ ]:
movies.head()

In [ ]:
ratings.head()

In [ ]:
ratings.isnull().sum()

In [ ]:
ratings.drop(columns="timestamp",axis=1, inplace=True)

In [ ]:
import matplotlib.pyplot as plt
plt.figure()
plt.hist(ratings["rating"],bins=50)
plt.xlabel("ratings")
plt.ylabel("count of ratings")
plt.show()


In [ ]:
ratings.head(10)

In [ ]:
user_counts = ratings.groupby("userId")["rating"].count()
user_counts.head()

In [ ]:
active_users = user_counts[user_counts >= 5].index
ratings_filtered = ratings[ratings["userId"].isin(active_users)]


In [ ]:
movie_counts = ratings_filtered.groupby("movieId")["rating"].count()
movie_counts.head()


In [ ]:
popular_movies = movie_counts[movie_counts >= 10].index
ratings_filtered = ratings_filtered[ratings_filtered["movieId"].isin(popular_movies)]
ratings_filtered

In [ ]:
!pip install scikit-surprise
from surprise import Dataset, Reader, SVD


In [ ]:
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(ratings_filtered[['userId', 'movieId', 'rating']], reader)


In [ ]:
from surprise.model_selection import train_test_split

trainset, testset = train_test_split(data, test_size=0.2, random_state=42)



In [ ]:
svd_model = SVD()
svd_model.fit(trainset)


In [ ]:
from surprise import accuracy

predictions = svd_model.test(testset)
rmse = accuracy.rmse(predictions)
print("Test RMSE:", rmse)



In [ ]:
# Example: recommend top 10 movies for userId = 1
user_id = 1
all_movie_ids = ratings_filtered['movieId'].unique()
rated_movies = ratings_filtered[ratings_filtered['userId'] == user_id]['movieId'].tolist()

# Filter out movies already rated
movies_to_predict = [m for m in all_movie_ids if m not in rated_movies]

# Predict ratings
predictions_user = [ (m, svd_model.predict(user_id, m).est) for m in movies_to_predict ]

# Sort by predicted rating descending
predictions_user.sort(key=lambda x: x[1], reverse=True)

# Take top 10
top_10 = predictions_user[:10]

# Optional: convert to DataFrame
recommended_df = pd.DataFrame(top_10, columns=['movieId', 'predicted_rating'])

# Merge with movies_final to get titles
recommended_df = recommended_df.merge(movies[['movieId','title','genres']], on='movieId')
recommended_df


In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(6,4))
plt.hist(ratings_filtered["rating"], bins=20)
plt.xlabel("Rating")
plt.ylabel("Count")
plt.title("Distribution of Movie Ratings")
plt.show()


In [ ]:
# Split genres and count
genres = recommended_df["genres"].str.split("|").explode()
genre_counts = genres.value_counts().head(10)

plt.figure(figsize=(7,4))
genre_counts.plot(kind="bar")
plt.xlabel("Genre")
plt.ylabel("Count")
plt.title("Top Genres in Recommendations")
plt.show()


In [ ]:
plt.figure(figsize=(6,4))
plt.hist(recommended_df["predicted_rating"], bins=10)
plt.xlabel("Predicted Rating")
plt.ylabel("Count")
plt.title("Predicted Rating Distribution")
plt.show()


In [ ]:
recommended_df.to_csv("movie_recommendations.csv", index=False)


In [ ]:
import os

# Replace these with your actual AWS credentials
os.environ["AWS_ACCESS_KEY_ID"] = "YOUR_ACCESS_KEY"
os.environ["AWS_SECRET_ACCESS_KEY"] = "YOUR_SECRET_ACCESS_KEY"
os.environ["AWS_DEFAULT_REGION"] = "us-east-1"


In [ ]:
import os

print(os.environ.get("AWS_ACCESS_KEY_ID"))
print(os.environ.get("AWS_SECRET_ACCESS_KEY"))


In [ ]:
import os
import boto3

# Set AWS credentials
os.environ["AWS_ACCESS_KEY_ID"] = "YOUR_ACCESS_KEY"
os.environ["AWS_SECRET_ACCESS_KEY"] = "YOUR_SECRET_ACCESS_KEY"
os.environ["AWS_DEFAULT_REGION"] = "us-east-1"

# Create S3 client
s3 = boto3.client("s3")

# Test
print(s3.list_buckets()["Buckets"])


In [ ]:
import boto3

s3 = boto3.client("s3")

s3.upload_file(
    "movie_recommendations.csv",
    "movie-reco-processed-nishu",
    "recommendations/movie_recommendations.csv"
)

print("Recommendations uploaded to S3 successfully")
